<a href="https://colab.research.google.com/github/Dan2912/projeto_fort_tech/blob/main/pepiline_s3_bronze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##IP-Colab

In [ ]:
import requests
print("IP do Colab:", requests.get('https://ifconfig.me').text)


IP do Colab: 35.221.229.245


##Instalação da biblioteca

In [ ]:
pip install boto3 psycopg2-binary pandas pyarrow s3fs


INFO: pip is looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/

##Código (RDS + leitura do S3 parquet)

In [ ]:
import pandas as pd
import psycopg2
import boto3
import s3fs
from getpass import getpass
from datetime import date

# === ACESSO AO RDS ===
host = input("Host do RDS: ")
database = input("Nome do banco: ")
user = input("Usuário do RDS: ")
password = getpass("Senha do RDS: ")

# Conectar ao RDS
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port='5432'
)

# Ler dados da tabela vendas (pode ser tudo ou só os últimos dias)
query = "SELECT * FROM vendas"
df_rds = pd.read_sql(query, conn)
conn.close()

# === ACESSO AO S3 ===
aws_access_key = getpass("AWS Access Key ID: ")
aws_secret_key = getpass("AWS Secret Access Key: ")
bucket = input("Bucket do S3: ").strip()
caminho_parquet = "bronze/base_vendas/carga_inicial/vendas_01.parquet"

# Criar sistema de arquivos S3 com boto3 + s3fs
fs = s3fs.S3FileSystem(
    key=aws_access_key,
    secret=aws_secret_key
)

# Ler parquet direto do S3
df_s3 = pd.read_parquet(f"s3://{bucket}/{caminho_parquet}", filesystem=fs)

print(f"🔎 Registros no RDS: {len(df_rds)}")
print(f"📦 Registros no parquet S3: {len(df_s3)}")


Host do RDS: fort-tech.c6l8w0a8uczb.us-east-1.rds.amazonaws.com
Nome do banco: postgres
Usuário do RDS: postgres
Senha do RDS: ··········


/tmp/ipython-input-1-45135788.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rds = pd.read_sql(query, conn)


AWS Access Key ID: ··········
AWS Secret Access Key: ··········
Bucket do S3: data-lake-eletro-fort
🔎 Registros no RDS: 100
📦 Registros no parquet S3: 50


##🔍 Etapa 3: Comparar os id_venda e filtrar só os novos

In [ ]:
# Identificar IDs que já existem no parquet
ids_existentes = set(df_s3['id_venda'])

# Filtrar novos registros do RDS
df_novos = df_rds[~df_rds['id_venda'].isin(ids_existentes)]

print(f"🆕 Novos registros encontrados: {len(df_novos)}")


🆕 Novos registros encontrados: 50


##💾 Etapa 4: Salvar os novos dados no S3

In [ ]:
if len(df_novos) > 0:
    hoje = date.today().isoformat()
    novo_arquivo = f"bronze/base_vendas/data={hoje}/vendas_{hoje}.parquet"

    df_novos.to_parquet(
        f"s3://{bucket}/{novo_arquivo}",
        index=False,
        engine='pyarrow',
        filesystem=fs
    )
    print(f"✅ Novos dados salvos em: s3://{bucket}/{novo_arquivo}")
else:
    print("✅ Nenhum novo registro para salvar.")


✅ Novos dados salvos em: s3://data-lake-eletro-fort/bronze/base_vendas/data=2025-06-21/vendas_2025-06-21.parquet
